### 策略网络的实现

2 层的全连接网络

第一层：

长度为 4 的向量转换为长度为 128 的向量;

第二层：

长度为 128 的向量转换为 2 的向量;


![](img/01.jpg)


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers, optimizers

In [ ]:
learning_rate = 0.0002
gamma = 0.98

In [ ]:
class Policy(keras.Model):
    def __init__(self):
        super(Policy, self).__init__()
        self.data = []
        self.data = [] # 存储轨迹
        # Kaiming初始化，也称之为He初始化，又称之为MSRA初始化，出自何凯明(Kaiming He)
        # 正态化的kaiming初始化——he_normal
        self.fc1 = layers.Dense(128, kernel_initializer='he_normal')
        self.fc2 = layers.Dense(2, kernel_initializer='he_normal')
        # 网络优化器
        self.optimizer = optimizers.Adam(learning_rate=learning_rate)
        def call(self, inputs, training=None):
            # 输入状态 s [4]
            x = tf.nn.relu(self.fc1(inputs))
            x = tf.nn.relu(self.fc2(x), axis=1)
            return x

        def train_net(self, tape):
            R = 0 # 初始奖励为 0
            for r, log_prob in self.data[::-1]:
                R = r + gamma * R # 每个时间戳上的回报
                loss = -log_prob * R
                with tape.stop_recording():
                    grads = tape.gradient(loss, self.trainable_variables)
                    self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
            self.data = []

In [ ]:
env = gym.make('CartPole-v1')
def main():
    pi = Policy()
    pi(tf.random.normal((4, 4)))
    pi.summary()
    score = 0.0
    print_interval = 20 # 打印间隔
    returns = []

    for n_epi in range(400):
        s = env.reset() # 回到初始状态
        